In [ ]:
"""
This is legacy code written for Python 2.7

ODS codes are formal ID numbers of all NHS-funded institutions in England.
Adding ODS codes to the dataset is useful in that they can be used to merge comments data with
other datasets. Unfortunately, the NHS Choices database relies on an internal ID numbering
sheme which is unrelated to the ODS codes.

Please take note that the apikey JROLO used here is not a proper API Key. You need to apply
to NHS Choices to obtain your own API key.

"""

import requests
import os
import sys
import csv

# download data about all commented-on GP practices

lista_gp = []

# create list of GP practices
with open("final2016.csv", "rb") as wsad:
    reader = csv.reader(wsad, delimiter=",")
    kolumna = 0
    kol_typ_kom = 0
    typ_insttt = 0
    for i, line in enumerate(reader):
        if i != 0:
            if line[typ_insttt] == "gppractices":
                if len(line[kolumna]) > 0:
                    r = line[kolumna]
                    fin = r.index('gppractices/') + len("gppractices/")
                    if r[fin:] not in lista_gp:
                        if line[kol_typ_kom] == "comment":
                            lista_gp.append(r[fin:])
        else:

            for ind, wart in enumerate(line):
                if wart == "instttn.http.address":
                    kolumna = ind
                elif wart == "mssg.type":
                    kol_typ_kom = ind
                elif wart == "instttn.type":
                    typ_insttt = ind

print "list of GP practices is now complete"
# print "lista przychodni zrobiona"

os.chdir("c:\users\user\desktop\gp_data_folder") 

for przychodnia in lista_gp: 
    adres = "http://v1.syndication.nhschoices.nhs.uk/organisations/gppractices/" + przychodnia + "/overview?apikey=JROLO"
    nazwa_pliku = "gp_" + str(przychodnia) + "_info" + ".xml"
    reload(sys)
    sys.setdefaultencoding('UTF8')
    r = requests.get(adres)
    with open(nazwa_pliku, "wb") as f:
        f.write(r.text)
    reload(sys)


os.chdir("c:\users\user\desktop")

print "ok"

lista przychodni zrobiona


In [1]:


# -*- coding: utf-8 -*-
# przerabianie mojej bazy danych by wydobyć informacje z oryginalnego zestawu danych 
# + ściągnąć kod ODS
# dla komentarzy pacjentów/klientów na temat GP practices
# DANE BĘDĄ PRZYGOTOWANE DO OBRÓBKI PRZY POMOCY MODELU LDA2VEC

import csv
import os
import glob
import sys

os.chdir("c:\users\user\desktop") 

if os.path.isfile("revs_with_ods.csv"):
    os.remove("revs_with_ods.csv")

os.chdir("C:\Users\user\Desktop\modelowe_wypluwki\preprocess\przychodnie_info")

kodownik = {}
for plik in glob.glob("*.txt"):
    kod = plik[12:]
    kon = kod.find("_")
    klucz = kod[:kon]
    with open(plik, "rb") as d:
        tekst = d.read()
        test = sys.getsizeof(tekst)
        if test > 2000:
            a = tekst.find("<dt>ODS code:</dt>")
            b = a + len("<dt>ODS code:</dt>") + 4
            c = tekst[b:(b+20)]
            koniec = c.find("</dd>")
            wart = c[:koniec]
            kodownik[klucz] = wart
        else:
            kodownik[klucz] = "nan"

os.chdir("C:\Users\user\Desktop")


listka = [8,9,10,11,40,43,46,49,52,55]
with open("final2016.csv", "rb") as wsad:
    reader = csv.reader(wsad, delimiter=",")
    with open("revs_with_ods.csv", "ab") as dok:
        writer = csv.writer(dok, delimiter = ",") 
        for i, line in enumerate(reader):
            if i != 0:
                kielbasa = []
                if line[3] == "comment" and line[7] != "nan" and line[1] == "gppractices":
                    kielbasa.append(line[0])
                    kielbasa.append(line[1])
#                         TU WPISUJĘ KOD DO WSTAWIENIA SYGNATURY ODS
                    a = line[12].find("gppractices/")
                    b = a + len("gppractices/")
                    klucz = line[12][b:]
                    kielbasa.append(kodownik[klucz])
                    kielbasa.append(line[7])
#                     TUTAJ DODAJĘ DODATKOWE KOMÓRKI
                    if len(line[52]) != 0 or len(line[46]) != 0 or len(line[49]) != 0:
                        for pozycja in listka:
                            if len(line[pozycja]) != 0:
                                kielbasa.append(line[pozycja])
                            else:
                                kielbasa.append("nan")
                        writer.writerow(kielbasa)
                    else:
                        pass
                else:
                    pass
            else:
                kielbasa = []
                kielbasa.append(line[0])
                kielbasa.append(line[1])
                kielbasa.append("ODS.kod")
                kielbasa.append(line[7])
                for element in listka:
                    kielbasa.append(line[element])
                writer.writerow(kielbasa)


print
print "plik wybrane_dane_z_ods.csv stworzony z kodami ODS"


plik wybrane_dane_z_ods.csv stworzony z kodami ODS
